## import osparc and start application

In [ ]:

%load_ext autoreload
%autoreload 2

import importlib
import numpy as np
from s4l_v1._api.application import run_application, get_app_safe

if importlib.util.find_spec('osparc') is None:
    !pip install osparc
!python -c "import osparc; print(osparc.__version__)"

if get_app_safe() is None:
    run_application()
app = get_app_safe()


## other imports

In [ ]:

import os
import osparc

from dotenv import dotenv_values
from concurrent.futures import ThreadPoolExecutor
from file_utils import stage_simulation
from job_utils import create_and_upload_job, monitor_jobs, download_results


## load config file and read config vars

In [ ]:

# Configuration variables
runner_solver_name = "simcore/services/comp/s4l-python-runner-8-0-0"
runner_solver_version = "1.2.0"

electrode_range = range(0, 4)
project_name = "my_project.smash"
template_path = 'template.txt'

home_dir = '/home/smu/work/workspace'
code_dir = os.path.join(home_dir, 'my_code')
staging_dir = os.path.join(home_dir, 'staging')
result_dir = os.path.join(home_dir, 'results')

# Retrieve API credentials from separate file
config = dotenv_values("api.env")  # change file if user is different !!
cfg = osparc.Configuration(
        host=config["OSPARC_API_HOST"],      
        username=config['OSPARC_API_KEY'],
        password=config["OSPARC_API_SECRET"],
    )
print("pointing to ", cfg.host)

# Make sure directories exist
os.makedirs(staging_dir, exist_ok=True)
os.makedirs(result_dir, exist_ok=True)
os.makedirs(code_dir, exist_ok=True)


## stage and launch jobs

In [ ]:

%autoreload 2
# main execution logic
with osparc.ApiClient(cfg) as api_client:
    solvers_api = osparc.SolversApi(api_client)
    files_api = osparc.FilesApi(api_client)

    metadata = {}

    # using a thread pool for parallel job creation
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [
            executor.submit(
                create_and_upload_job,
                files_api, 
                solvers_api, 
                project_name, 
                e_index,
                stage_simulation(  
                    project_name, 
                    e_index, 
                    template_path, 
                    home_dir,
                    {
                        'template_vars': {
                            'smashfile': project_name,
                            'electrode_index': e_index
                        },
                        'file_suffix': 'em_sim',
                        'zip_files': [
                            'requirements.txt',  # Additional files
                            'emconfig.json',
                            os.path.join(home_dir, project_name)  # Smashfile
                        ],
                        'zip_folders': [
                            code_dir  # Folders to include
                        ],
                        'zipped_filename': f'em_sim_{project_name}_E{str(e_index).zfill(2)}.zip'
                    }
                ),  # This call returns the path to the staged and zipped simulation
                runner_solver_name, 
                runner_solver_version
            )
            for e_index in electrode_range  # Looping over the electrode range to submit jobs in parallel
        ]
        
        # Collect results from the futures
        for future in futures:
            result = future.result()  # Wait for each future to complete
            metadata[result[0]] = result[1]  # Store metadata (e.g., job ID and status)

    # Monitor job statuses after all jobs are created
    statuses = monitor_jobs(solvers_api, metadata, runner_solver_version)

    # Download and extract the results after job completion
    download_results(files_api, solvers_api, metadata, runner_solver_version, result_dir)
